# Importing necessary packages

In [2]:
import pandas as pd
import numpy as np
# import cPickle
import _pickle as cPickle 
from gensim.models import word2vec
import logging
import gensim
from sklearn.metrics.pairwise import cosine_distances,cosine_similarity
print("finished importing")

c:\anaconda\envs\machine_learning\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\anaconda\envs\machine_learning\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\anaconda\envs\machine_learning\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


finished importing


# Preprocessing

#data used in this <br>
#prep_data_underscore,mustHaveSkills-2.csv,naukri_skill_full

# Prep_data_tokens

In [3]:
with open("prep_data_tokens_underscore_1", "rb") as g:
    data_dice = cPickle.load(g)
    print("Loaded sucessfully")
data_dice
len(data_dice) #data_dice contains set of skills

Loaded sucessfully


7146

# MustHaveskills data

In [3]:
import pandas as pd

c:\anaconda\envs\machine_learning\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\anaconda\envs\machine_learning\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\anaconda\envs\machine_learning\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [6]:
data_must_have = pd.read_csv('mustHaveSkills-2.csv',header = 0, encoding='ISO-8859-1') #reading the dataset
del data_must_have['job_brief_id'] #droping job_breif_id because this column doesn't have any relevance

data_must_have.drop_duplicates(subset=['keyword_name', 'job_title'], keep='last') #drop the records which has same keyword and job_title 
print(data_must_have.shape) 

data_must_have = data_must_have[data_must_have["job_title"] != 0] #take only the records which doesn't have zero for the column job_title



(261724, 2)


In [16]:
len(data_must_have["job_title"].unique())

4596

In [18]:
data_must_have.drop_duplicates()
len(data_must_have["job_title"].unique())

4596

In [8]:
data_must_have['keyword_name'] = data_must_have['keyword_name'].str.lower() # coverting keyword_name column to lowercase
data_must_have['keyword_name'] = data_must_have['keyword_name'].str.replace(' ' ,'_') # replacing blank spaces with underscore in keyword_name column
print("Finished")
# data_must_have['keyword_name'] = data_must_have['keyword_name'].astype(unicode)
# data_must_have['keyword_name'] = data_must_have['keyword_name'].apply(lambda x: x.encode('utf-8'))

Finished


In [8]:
data_must_have

,keyword_name,job_title
0,php,Web Designer
1,php,Web Developer
2,php,Full Stack Developer
3,php,Dev Ops
4,php,Product Manager
...,...,...
9995,deliverables,Senior Producer
9996,budget_management,Producer
9997,budget_management,Senior Producer
9998,digital_marketing,Digital Marketing


In [9]:
data_must_have['job_title'] = data_must_have['job_title'].str.lower()
print("finished")

finished


In [10]:
data_must_have.tail(5)

,keyword_name,job_title
9995,deliverables,senior producer
9996,budget_management,producer
9997,budget_management,senior producer
9998,digital_marketing,digital marketing
9999,digital_marketing,product management


In [10]:
gr_df_keywordname = data_must_have.groupby('keyword_name')['job_title'].apply(list) #group the dataset based on keyword_name 
gr_df_jobtitle = data_must_have.groupby('job_title')['keyword_name'].apply(list) #group the dataset based on keyword_name 
print("finished")

finished


In [13]:
len(gr_df_keywordname)


1020

In [14]:
must_have_data = [] #contains the skills set for all available roles after the iteration
for vector_list in gr_df_jobtitle:
    xx = list(set(vector_list))  
    if xx not in must_have_data: # eliminating repeatations 
        must_have_data.append(xx)
must_have_data[:2]

[['.net'], ['encryption_software']]

# Taking skills from naukri

import pandas as pd
data_naruki = pd.read_csv('naukri_skill_full', header = 0, encoding='ISO-8859-1')

data_naruki.loc[data_naruki["id"]==10]
data_naruki

len(data_naruki["skill"].unique()),len(data_naruki["id"].unique())

#drop duplicate
data_naruki.drop_duplicates(subset=['id', 'skill'], keep='last')
#lower-case
data_naruki['skill'] = data_naruki['skill'].str.lower()
data_naruki['skill'] = data_naruki['skill'].str.replace(' ','_')
data_naruki_final = data_naruki.groupby('id')['skill'].apply(list)
len(data_naruki_final),print("finished")

type(data_naruki_final),len(data_naruki_final),data_naruki_final[1]

type(data_dice),len(data_dice)

# Naukri finished

In [11]:
final_skill_set=[]
skills=[]
for skill_set in gr_df_jobtitle:
    if len(skill_set) > 2:
        temp=[]
        for skill in skill_set:
            if skill not in temp: #eliminating skill repeatations within skill set
                temp.append(skill)
                if skill not in skills: #used for counting the number of skills can be commented if not needed
                    skills.append(skill)
        if temp not in final_skill_set:
            final_skill_set.append(temp)

In [12]:
len(final_skill_set),len(skills)

(2721, 6843)

In [15]:
# appending the group skills(based on jobtitle from must_have_skills.csv) to data_dice_tmp
data_dice_tmp = data_dice
matchings = []
for must_have in gr_df_jobtitle:
    if len(must_have) > 2:
        if must_have not in data_dice:
            data_dice_tmp.append(must_have)
print("finiiiiiiiiiiiiiiiiiished")

finiiiiiiiiiiiiiiiiiished


#### prep_data_tokens and must have skills are combined 

In [16]:
data_final = []
for i in data_dice_tmp:
    vector =[]
    for j in i:
        if isinstance(j,str): #eliminating non strings
            if j not in vector: #eliminating repeataions within each skill set group
                vector.append(j)
    if i not in data_final:  #eliminaing skill sets repeatations
        data_final.append(vector)
data_final

[['addm\r',
  'configuration_management_database\r',
  'servicenow\r',
  'it_service_management\r',
  'itil\r',
  'service_management\r',
  'ddm\r',
  'bmc_remedy\r',
  'issue_tracking\r',
  'configuration\r',
  'knowledge_management\r',
  'help_desk\r',
  'normalization\r',
  'bppm\r',
  'patrol\r',
  'bmc_patrol\r',
  'event_management\r',
  'systems_management\r',
  'performance_management\r',
  'awr\r',
  'tkprof\r',
  'sql_trace\r',
  'data_pump\r',
  'oracle_dba\r',
  'oracle_grid_control\r',
  'statspack\r',
  'tde\r',
  'recovery\r',
  'mef\r',
  'vldb\r',
  'nmon\r'],
 ['adabas\r',
  'natural\r',
  'energy\r',
  'roscoe\r',
  'fiscal_management\r',
  'jcl\r',
  'vse\r',
  'entirex\r',
  'war\r',
  'jar\r',
  'webmethods\r',
  'root_cause_analysis\r',
  'cost_estimation\r',
  'assembly_language\r',
  'bal\r',
  'smp/e\r',
  'ipcs\r',
  'tpf\r',
  'pl/i\r',
  'ibm_utilities\r',
  'rexx\r',
  'syncsort\r',
  'clist\r',
  'smf\r',
  'panvalet\r',
  'tso\r',
  'ca-7\r',
  'endevor\

In [17]:
len(data_final),data_final[11]

(7390,
 ['acaps\r',
  'cacs\r',
  'collections\r',
  'pki\r',
  'cas\r',
  'productivity_improvement\r',
  'cfs\r',
  'hogan\r',
  'abend-aid\r',
  'changeman\r',
  'fileaid\r',
  'vsam\r',
  'zeke\r',
  'assembly_language\r',
  'bal\r',
  'smp/e\r',
  'ipcs\r',
  'tpf\r',
  'pl/i\r',
  'base24\r',
  'tal\r',
  'tandem\r',
  'scobol\r',
  'tacl\r',
  'atm\r',
  'cgi\r',
  'professional_development\r',
  'compensation\r',
  'business_process\r',
  'health_care\r',
  'accounting_software\r'])

In [ ]:
all_skill=[]
for skill_set in data_final:
    for skill in skill_set:
        if skill not in all_skill:
            all_skill.append(skill)
print("total skills ",len(all_skill),all_skill)
# pd.DataFrame(all_skill).to_csv("all_skills.csv")

total skills  ['addm', 'configuration_management_database', 'servicenow', 'it_service_management', 'itil', 'service_management', 'ddm', 'bmc_remedy', 'issue_tracking', 'configuration', 'knowledge_management', 'help_desk', 'normalization', 'bppm', 'patrol', 'bmc_patrol', 'event_management', 'systems_management', 'performance_management', 'awr', 'tkprof', 'sql_trace', 'data_pump', 'oracle_dba', 'oracle_grid_control', 'statspack', 'tde', 'recovery', 'mef', 'vldb', 'nmon', 'adabas', 'natural', 'energy', 'roscoe', 'fiscal_management', 'jcl', 'vse', 'entirex', 'war', 'jar', 'webmethods', 'root_cause_analysis', 'cost_estimation', 'assembly_language', 'bal', 'smp/e', 'ipcs', 'tpf', 'pl/i', 'ibm_utilities', 'rexx', 'syncsort', 'clist', 'smf', 'panvalet', 'tso', 'ca-7', 'endevor', 'ca_ops/mvs', 'computer_operator', 'spf', 'acsls', 'netbackup', 'backup', 'backup_administration', 'vtl', 'emc_avamar', 'data_domain', 'storage+', 'emc_recoverpoint', 'emc_vmax', 'emc_isilon', 'nas', 'ndmp', 'beyond_co

In [6]:
import pandas as pd

# Create a sample DataFrame with integer values
df = pd.DataFrame({'A': [1, 2, 3],
                   'B': [4, 5, 6]})

# Print the original DataFrame and its data types
print(df)
print(df.dtypes)

# Convert the values in column 'B' to floats using astype()
df['B'] = df['B'].astype(str)

# Print the updated DataFrame and its data types
print(df)
print(df.dtypes)


   A  B
0  1  4
1  2  5
2  3  6
A    int64
B    int64
dtype: object
   A  B
0  1  4
1  2  5
2  3  6
A     int64
B    object
dtype: object
